In [ ]:
import simpleFoam_tools as sft
import kphi
import pandas as pd
import numpy as np

In [ ]:
remove = 1
if remove:
    sft.remove_run_files(".")

In [ ]:
domain_name = "cylinder"
vti_path = f"constant/geometry/{domain_name}.vti"
stl_path = f"constant/triSurface/{domain_name}.stl"

In [ ]:
radii = 10
shape = (100, 50, 50)

In [ ]:
sft.make_cylinder_vti(radii, shape[::-1], vti_path)

In [ ]:
scale=1e-6
dp = 1e-3
stl = f"{domain_name}.stl"
refinement = 1
factor_mesh = 1
location_in_mesh = (25, 25, 25)
# location_in_mesh = (50, 70, 20)
boundary_type = "Wall" # "Wall" or "symmetryPlane"
end_time = 300
write_interval = 50

In [ ]:
sft.make_stl(vti_path, stl_path)

In [ ]:
mesh_resolution = tuple(x * factor_mesh for x in shape)
blockMeshDict_path    = "system/blockMeshDict"
controlDict_path      = "system/controlDict"
p_field_path          = "0/p"
U_field_path          = "0/U"
snappyHexMeshDict_path = "system/snappyHexMeshDict"
sft.generate_blockMeshDict(shape, mesh_resolution, blockMeshDict_path, boundary=boundary_type)
sft.generate_snappyHexMeshDict(location_in_mesh, stl, snappyHexMeshDict_path, refinement=refinement)
sft.generate_controlDict(controlDict_path, end_time=end_time, write_interval=write_interval)
sft.generate_pressure_field(p_field_path, dp=dp, boundary=boundary_type)
sft.generate_velocity_field(U_field_path, boundary=boundary_type)

In [ ]:
sft.run_simplefoam(".", scale=scale)

In [ ]:
df_q = pd.read_csv("q_in.csv")
df_q.head()

In [ ]:
phi = kphi.phi_vti(vti_path)
print(f"porosity: {phi:.02%}")

In [ ]:
A = shape[1] * shape[2] * scale**2
miu = 1e-3
L = shape[0] * scale
Q = np.abs(df_q["flowRate_phi"].iloc[-1])  
dp = dp * 1e3 # convert to Pa
k = Q * miu * L / (A * dp)
print(f"permeability: {k:.02e} m^2")
print(f"permeability: {k * 1.01324997e15:.02f} md")

In [ ]:
k_analytical = kphi.k_cylinder(radii*scale)
A_tube = np.pi * (radii * scale)**2
k_analytical = k_analytical * A_tube / A
print(f"analytical permeability: {k_analytical:.02e} m^2")
print(f"analytical permeability: {k_analytical * 1.01324997e15:.02f} md")
print(f"relative error: {(k - k_analytical) / k_analytical:.02%}")

In [ ]:
A_mesh = df_q["area"].iloc[-1]
rel_error = np.abs(A_tube - A_mesh) / A_tube
print(f"mesh area: {A_mesh:.02e} m^2")
print(f"analytical area: {A_tube:.02e} m^2")
print(f"relative error in mesh area: {rel_error:.02%}")